# Etapa de Pré-processamento: Extração de Embeddings de Imagens de Daninhas

## 📚 Descrição
Esta etapa do pipeline de processamento é responsável por receber imagens de plantas daninhas, realizar o pré-processamento necessário e extrair **embeddings** utilizando um modelo pré-treinado. Os embeddings servirão como vetores de características que serão utilizados nas etapas posteriores de classificação e análise.

---

## 🔧 Etapas do Pré-processamento
1. **Carregamento das Imagens:**  
   - As imagens são carregadas a partir de um diretório especificado.
   - Caso as imagens não estejam em um espaço de cores RGB elas serão convertidas
   - Cada imagem é redimensionada para o tamanho esperado pelo modelo pré-treinado (por exemplo, 299x299 pixels para "imagenet").
   
2. **Normalização das Imagens:**  
   - Os valores dos pixels são escalados para o intervalo [0, 1] ou normalizados conforme os requisitos do modelo.

3. **Data Augmentation (Opcional):**  
   - Técnicas como rotação, flip horizontal/vertical e ajuste de brilho são aplicadas para aumentar a variabilidade do conjunto de dados.

4. **Extração de Embeddings:**  
   - Um modelo pré-treinado (imagenet) é carregado sem a camada de classificação.
   - As imagens são passadas pelo modelo, e os embeddings (vetores de características) são extraídos da penúltima camada.

5. **Tratamento dos Embeddings e balanceamento:**  
   - O arquivo vetorial é salvo em uma planilha com as caracteristicas e sua respectiva classe.
   - caso necessário para otimizar o treinamento pode ser feito um balanceamento por oversampling aumentando as amostras de menor número.
   

---

In [1]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tqdm.notebook import tqdm
tqdm().pandas()
from tensorflow.keras.optimizers import *
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN

0it [00:00, ?it/s]

In [14]:
#caso suas imagens não esteja em rgb
def convert_images_to_rgb(root_folder):
    """
    Converte todas as imagens em um diretório e seus subdiretórios para o formato RGB.
    
    Args:
        root_folder (str): Caminho para a pasta raiz contendo os subdiretórios com as imagens.
    """
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                with Image.open(file_path) as img:
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        img.save(file_path)
                        print(f'Convertido para RGB: {file_path}')
                    else:
                        print(f'Já está em RGB: {file_path}')
            except Exception as e:
                print(f'Erro ao processar {file_path}: {e}')

# Caminho para a pasta raiz
root_folder = "./caminho das imagens aqui"
convert_images_to_rgb(root_folder)

Já está em RGB: ./Matrice_png\Daninha\ANDIRA_1.png
Já está em RGB: ./Matrice_png\Daninha\ANDIRA_2.png
Já está em RGB: ./Matrice_png\Daninha\ANDIRA_3.png
Já está em RGB: ./Matrice_png\Daninha\ANDIRA_4.png
Já está em RGB: ./Matrice_png\Daninha\ANDIRA_5.png
Já está em RGB: ./Matrice_png\Daninha\ANDIRA_6.png
Já está em RGB: ./Matrice_png\Daninha\ANDIRA_7.png
Já está em RGB: ./Matrice_png\Daninha\ANDIRA_8.png
Já está em RGB: ./Matrice_png\Daninha\ANDIRA_9.png
Já está em RGB: ./Matrice_png\Daninha\ERIOBOTRYA_1.png
Já está em RGB: ./Matrice_png\Daninha\ERYTHRINA_1.png
Já está em RGB: ./Matrice_png\Daninha\HANDROANTHUS_1.png
Já está em RGB: ./Matrice_png\Daninha\ORMOSIA_HENRY_1.png
Já está em RGB: ./Matrice_png\Daninha\ORMOSIA_HENRY_2.png
Já está em RGB: ./Matrice_png\Daninha\ORMOSIA_HENRY_3.png
Já está em RGB: ./Matrice_png\Daninha\psidium_cattle_1.png
Já está em RGB: ./Matrice_png\Daninha\psidium_guajav_1.png
Já está em RGB: ./Matrice_png\Daninha\spathodea_1.png
Já está em RGB: ./Matrice_png

In [15]:
# Diretório das imagens
dataset_path = 'caminho das imagens aqui'

# Função para carregar imagens e rótulos
def load_dataset(dataset_path, target_size=(299, 299)):
    data = []
    labels = []
    classes = os.listdir(dataset_path)
    
    for class_label in classes:
        class_path = os.path.join(dataset_path, class_label)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = Image.open(img_path).resize(target_size)
                img = np.array(img)
                data.append(img)
                labels.append(class_label)
    
    return np.array(data), np.array(labels)

In [16]:

X, y = load_dataset(dataset_path)

In [17]:
# Inicializar o modelo base
base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

In [18]:

# Inicializar o gerador de dados com augmentação
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Pré-processar as imagens
X_preprocessed = preprocess_input(X)

In [19]:

# Gerar embeddings para imagens aumentadas
embeddings = []
labels = []

for i in range(len(X_preprocessed)):
    img = X_preprocessed[i]
    label = y[i]
    
    img = np.expand_dims(img, axis=0)
    
    for batch in datagen.flow(img, batch_size=1):
        augmented_img = batch[0]
        augmented_img = np.expand_dims(augmented_img, axis=0)
        embedding = model.predict(augmented_img)
        
        embeddings.append(embedding[0])
        labels.append(label)
        
        # Gerar 5 aumentos por imagem
        if len(labels) >= (i+1) * 5:
            break

# Converter para DataFrame
embeddings_df = pd.DataFrame({
    'Classe': labels,
    'embedding': list(embeddings)
})

1/1 [==============================] - 0s 75ms/step


In [20]:
# Salvar o DataFrame em um CSV
embeddings_df.to_csv('embeddings_with_augmentation_matrice.csv', index=False)

In [21]:
np.savez_compressed('embeddings_with_augmentation_matrice.npz', embeddings=embeddings, labels=y)

In [22]:
print(embeddings_df.head())

    Classe                                          embedding
0  Daninha  [0.15025716, 0.1363607, 0.11102749, 0.19376248...
1  Daninha  [0.034448422, 0.10048885, 0.17525256, 0.136251...
2  Daninha  [0.028678495, 0.14451991, 0.01997426, 0.405101...
3  Daninha  [0.21235135, 0.22944842, 0.12351295, 0.2267802...
4  Daninha  [0.08023298, 0.1601995, 0.05822358, 0.5506959,...


In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

# Carregar os dados
file_path = './embeddings_with_augmentation_matrice.csv'
data = pd.read_csv(file_path)

# Corrigir formatação e converter embeddings para vetores NumPy
def fix_and_convert_embedding(embedding_str):
    # Remove leading/trailing whitespace and brackets
    embedding_str = embedding_str.strip('[]')
    
    # Convert the string to a NumPy array
    values = np.fromstring(embedding_str, sep=' ')
    
    return values

def preprocess_embeddings(data):
    data['embedding'] = data['embedding'].apply(fix_and_convert_embedding)
    return data

# Aplicar SMOTE para balancear classes
def apply_smote(data, target_column, embedding_column):
    # Separar as features (X) e o alvo (y)
    X = np.stack(data[embedding_column].values)
    y = data[target_column]
    
    # Aplicar SMOTE
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    
    # Retornar os dados balanceados como um novo DataFrame
    balanced_data = pd.DataFrame({
        target_column: y_resampled,
        embedding_column: list(X_resampled)
    })
    return balanced_data

# Processar os dados e aplicar SMOTE
data = preprocess_embeddings(data)
balanced_data = apply_smote(data, target_column="Classe", embedding_column="embedding")

# Salvar o resultado em um novo arquivo CSV
output_path = 'balanced_embeddings_matrice.csv'
balanced_data.to_csv(output_path, index=False)

print(f"Arquivo balanceado salvo em: {output_path}")